In [2]:
from dotenv import load_dotenv
import pandas as pd 
import alpaca_trade_api as tradeapi
from prophet import Prophet
import os
from pathlib import Path
from pandas.tseries.offsets import DateOffset
import datetime as dt
from prophet import Prophet
import hvplot.pandas
import warnings
warnings.filterwarnings("ignore")


#working on ALPACA api's
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)



# Creating a start date, and TimeFrame
today = dt.date.today()
start = today - DateOffset(years=5)

start_date = start.date()
end_date = today.isoformat()
timeframe = "1Day"



# initializing the alpaca
tickers = ["AAPL", "TSLA", "MSFT"] #this will be gathered from sidebar eventually
tickers_dfs = []

df_portfolio_year = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date
).df

df_portfolio_year.index=pd.to_datetime(df_portfolio_year.index).date

# Initializing facebook prophet library
def forecast_next_day(tickers):
    results_df = pd.DataFrame()
    for ticker in tickers:
        stock_df = pd.DataFrame(df_portfolio_year.loc[df_portfolio_year["symbol"] == ticker]["close"])
        stock_df = stock_df.reset_index().rename(columns={"index": "ds", "close": "y"})
        model = Prophet()
        model.fit(stock_df)
        future = model.make_future_dataframe(periods=1, freq="D")
        forecast = model.predict(future)
        forecast_yhats= forecast[["yhat","yhat_lower","yhat_upper"]].tail(1).values[0]
        results_df = results_df.append({'ticker': ticker, 'yhat': forecast_yhats[0], 
                                        'yhat_lower': forecast_yhats[1], 'yhat_upper': forecast_yhats[2]}, 
                                        ignore_index=True)
    return results_df

# Call the function with the tickers list

results_df = forecast_next_day(tickers)

print(results_df)

#Calling the def fubction
#user = input("which ticker you want to forecast?") #this will be a drop down menu 
#print(forecast_next_day(user))

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


  ticker        yhat  yhat_lower  yhat_upper
0   AAPL  136.221685   93.525172  179.691234
1   TSLA   27.759964 -179.122974  249.769441
2   MSFT  241.771130  229.333674  253.043385


In [ ]:
from dotenv import load_dotenv
import pandas as pd 
import alpaca_trade_api as tradeapi
from prophet import Prophet
import os
from pathlib import Path
from pandas.tseries.offsets import DateOffset
import datetime as dt
from prophet import Prophet
import hvplot.pandas
import sys
import os

load_dotenv()

os.chdir("../Kunal")
sys.path.append(os.getcwd())
from app import PriceSummary

#working on ALPACA api's
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

print(alpaca_secret_key)
print(alpaca_api_key)

alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)



# Creating a start date, and TimeFrame
today = dt.date.today()
start = today - DateOffset(years=5)

start_date = start.date()
end_date = today.isoformat()
timeframe = "1Day"



# initializing the alpaca
tickers = ["AAPL", "TSLA", "MSFT"] #this will be gathered from sidebar eventually
tickers_dfs = []

df_portfolio_year = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date
).df

df_portfolio_year.index=pd.to_datetime(df_portfolio_year.index).date

results_df = pd.DataFrame()

# Initializing facebook prophet library
def forecast_next_day(ticker,inputdata):
    results_df = pd.DataFrame()
    for ticker in tickers:
        stock_df = pd.DataFrame(inputdata.loc[inputdata["symbol"] == ticker]["close"])
        stock_df = stock_df.reset_index().rename(columns={"index": "ds", "close": "y"})
        model = Prophet()
        model.fit(stock_df)
        future = model.make_future_dataframe(periods=1, freq="D")
        forecast = model.predict(future)
        forecast_yhats= forecast[["yhat","yhat_lower","yhat_upper"]].tail(1).values[0]
        results_df = results_df.append({'ticker': ticker, 'yhat': forecast_yhats[0], 
                                        'yhat_lower': forecast_yhats[1], 'yhat_upper': forecast_yhats[2]}, 
                                        ignore_index=True)
    return results_df

results_df = forecast_next_day(tickers,df_portfolio_year)

df = PriceSummary(df_portfolio_year)

print(results_df)
print(df)